In [ ]:
# pip install mysql-connector-python
# pip install sqlalchemy

In [ ]:
import pandas as pd
import sqlalchemy
import mysql.connector

In [ ]:
database_username = 'root'
database_password = 'Password'
database_ip       = 'localhost'
database_name     = 'mydb'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [ ]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Young$123!",
    database="mydb")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE Weather (TMAX FLOAT(10), TMIN FLOAT(10))")

In [ ]:
import requests
import pandas as pd
start_date, end_date = '2020-01-01', '2020-01-31'
start_goal = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m-%d')
end_goal = pd.date_range(start=start_date, end=end_date, freq='M') .strftime('%Y-%m-%d')

token = "Password"
url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
data  = []
for i in range(0,start_goal.size):
    start = start_goal[i]
    end = end_goal[i]
    params = {
        "datasetid": "GHCND",
        "stationid": "GHCND:USC00215638",
        "units": "standard",
        "startdate": start,
        "enddate": end,
        "limit": 1_000}
    response = requests.get(url, params=params, headers={"Token": token}).json()
    data.append(response)
    working_data = pd.json_normalize(data, record_path='results')

In [ ]:
df = working_data.drop(columns=['station', 'attributes'])
df = df.pivot(index='date', columns='datatype', values='value')

In [ ]:
df2 = df.drop(columns=['TOBS','PRCP','SN31','SN33','SNOW','SNWD','SX31','SX33'])
df2

In [ ]:
df2.to_sql('Weather', con=database_connection, if_exists='append', index=False)

In [ ]:
df2['date type'] = df2['date'].dt.strftime('%Y/%m/%d')

In [ ]:
mydb.close
mycursor.close()